This Notebook will setup a docker container registry. Once setup, it can be used from a remote docker client. But, no security was configured, so this is the insecured method. An example on how a remote docker client can use it is at the end of this notebook. This works on the Github Codespaces and a Linux VM. It does NOT work on the Kaggle or Google Colab.

In [ ]:
# Delete all docker images and instances.
!docker rm -f $(docker ps -aq)
!docker rmi -f $(docker images -aq)
!echo "-----------"
!docker ps -a
!echo ""
!docker image ls

In [ ]:
# Create and start the docker container registry.
!docker run -d -p 5000:5000 --name registry --restart=always registry
!echo "-----------"
!docker image ls
!echo ""
!docker ps -a

In [ ]:
# Locally test the docker container registry.
!curl http://localhost:5000/v2/_catalog
!docker pull ubuntu
!docker tag ubuntu localhost:5000/my-image-3825
!docker push localhost:5000/my-image-3825

In [ ]:
# Find the URL the remote docker clients should use.
import os
import subprocess

name = os.environ.get("CODESPACE_NAME")
domain = os.environ.get("GITHUB_CODESPACES_PORT_FORWARDING_DOMAIN")
port = subprocess.run("docker port registry | sed 's/.*://' | head -n 1", shell=True, capture_output=True, text=True).stdout.strip()
url = f"{name}-{port}.{domain}"
os.environ["DCR_URL"] = url
!echo "# Below is the CLI command the remote docker clients should use."
!echo "export DCR_URL=$DCR_URL"

This Docker Container Registry is all set up and working. It was tested with commands executed locally. Here are CLI commands to test it from a remote docker client.
It also shows how a docker client can use it.<br>
```
    % export DCR_URL=<use the URL shown above>
    	# Instead of the above URL address, use the one specific to your GitHub Codespaces session.
        # Do NOT include the 'http://' or 'https://' part.
    % curl https://$DCR_URL/v2/_catalog
        # Verify you see the image 'my-image-3825'.
    % docker tag ubuntu $DCR_URL/my-image-3825
        # Change tag such that the URL ($DCR_URL) points to your docker registry.
    % docker push $DCR_URL/my-image-3825
        # Push this to your docker registry.
    % docker pull ubuntu
        # Pull from docker.com.
    % docker tag ubuntu $DCR_URL/my-image-5031
        # Change tag such that the URL ($DCR_URL) points to your docker registry.
    % docker push $DCR_URL/my-image-5031
        # Push this to your docker registry.
    % curl https://$DCR_URL/v2/_catalog
        # Verify you see the image 'my-image-5031'.
```

In [ ]:
# The GitHub Codespaces will idle timeout and your docker setup will be deleted. To prevent this, you need to create a keepalive.
# But, even with this, there is a hard time limit. The default idle timeout is 30 minutes. The maximum runtime length is unknown due to a lack of info.
import time
while True:
    time.sleep(5*60)
    print(".", end="")